In [1]:
from eumap.misc import find_files, nan_percentile
from eumap.raster import read_rasters, save_rasters, write_new_raster
from eumap.parallel import TilingProcessing

import time as tm
import numpy as np
import os
# import ee
# import geemap
import multiprocessing
from eumap.raster import read_rasters
import rasterio
from rasterio import features
import geopandas as gpd
from osgeo import ogr, gdal
from multiprocessing import Pool
import warnings
from os.path import exists
import pandas as pd
from rasterio.features import rasterize

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
nut = gpd.read_file('/mnt/inca/EU_landmask/nut.shp')
nut = nut.loc[nut['LEVL_CODE']==3]
nut.reset_index(inplace=True)
nut['index']=nut.index
shapes = [(geometry, index) for geometry, index in zip(nut.geometry, nut.index)]

code = pd.read_csv('/mnt/inca/EU_landmask/iso_3366_1_country_code.csv')
code.loc[code['alpha-2']=='GR','alpha-2']='EL'
code.loc[code['alpha-2']=='GB','alpha-2']='UK'

input_files = find_files('/mnt/inca/EU_landmask/landmask2','*.tif')
input_files = [str(i) for i in input_files]
files = [(shapes,i) for i in input_files]

In [3]:
def worker_nut(shapes,ref):
    mask_ds = rasterio.open(ref)
    mask = np.zeros(mask_ds.shape)
    out_f = '/mnt/inca/EU_landmask/landmask_nuts/nutmask_EU' + ref[43::]
    
    rasterize(shapes, out=mask, transform=mask_ds.transform)
    save_rasters(ref, [out_f], mask, dtype='float32' ,n_jobs=30)
    return

In [4]:
# def worker_nut(nut,code,f):
#     mask_ds = rasterio.open(f)
#     out_f = '/mnt/inca/EU_landmask/landmask_nuts/nutmask_EU' + f[43::]
#     if exists(out_f):
#         return None
    
#     # Create raster
#     mask = np.zeros(mask_ds.shape)
    
#     for i in range(len(nut)):
#         print(i)
#         value = nut.iloc[i]['index']
#         isnut = features.geometry_mask([nut.iloc[i]['geometry']], out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
#         isnut = ~isnut
#         if np.any(isnut):
#             mask[isnut] = value
        
# #     if ~np.any(is_inland):
# #         a = f.split('_')[-2]
# #         print(f'{a} out of scope')
# #         return a
# #     out_t = '/mnt/inca/EU_landmask/landmask/test_inland' + f[55::]
# #     save_rasters(f, [out_t], is_inland, n_jobs=30)
# #     print(np.unique(mask))
#     save_rasters(f, [out_f], mask, dtype='float32' ,n_jobs=30)
#     return None

In [5]:
# test = ['/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N54E006_Map.tif'] #'/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N51E006_Map.tif','/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N48E000_Map.tif'

# for i in test:
#     result = worker_nut(nut,code,i)

with Pool(12) as p:
    result = p.starmap(worker_nut, files)

In [2]:
vrt_file = '/mnt/inca/EU_landmask/landmask_nuts/nuts_landmask.vrt'
# input_files = find_files('/mnt/inca/EU_landmask/landmask_nuts/','*.tif')
# input_files = [str(i) for i in input_files]
# vrt_dataset = gdal.BuildVRT(vrt_file, input_files)
# vrt_dataset.FlushCache()

In [ ]:
tif_file = '/mnt/inca/EU_landmask/nuts_10m_landmask.tif'
ref_file = '/mnt/inca/EU_landmask/U2018_CHA1218_12_V2020_20u1.tif'
with rasterio.open(ref_file) as src:
    projection = src.crs
    transform = src.bounds
    
options = [
    f'BLOCKSIZE=1024',
    f'BIGTIFF=YES',
    f'COMPRESS=DEFLATE',
    f'NUM_THREADS=8',
    f'LEVEL=9',
    f'FORMAT=COG'
]
# gdal.SetConfigOption('CHECK_DISK_FREE_SPACE', 'FALSE')
gdal.Warp(tif_file, vrt_file, format='GTiff', outputBounds=transform, dstSRS=projection, xRes=10, yRes=-10, options=options)